In [74]:
import numpy as np
import pandas as pd

In [75]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [76]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA1_Tr/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [77]:
mineralogy = mineralogy.dropna()

In [78]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [79]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [80]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [81]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
6854,72.110994,0.266316,14.696717,0.542898,2.001099,0.041407,0.665204,1.482435,4.070729,4.122201
6855,71.208950,0.307729,14.950085,0.593100,2.237432,0.043509,0.838751,1.830527,4.074379,3.915540
6856,71.208488,0.308209,14.950880,0.599381,2.230242,0.043584,0.838415,1.826726,4.075818,3.918258
6857,71.206057,0.308743,14.952091,0.605349,2.224116,0.043658,0.838495,1.823994,4.077145,3.920353
6858,71.201188,0.309349,14.953838,0.610951,2.219250,0.043731,0.839095,1.822565,4.078344,3.921690


In [82]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
6854,72.110994,0.266316,14.696717,0.542898,2.001099,0.041407,0.665204,1.482435,4.070729,4.122201
6855,71.208950,0.307729,14.950085,0.593100,2.237432,0.043509,0.838751,1.830527,4.074379,3.915540
6856,71.208488,0.308209,14.950880,0.599381,2.230242,0.043584,0.838415,1.826726,4.075818,3.918258
6857,71.206057,0.308743,14.952091,0.605349,2.224116,0.043658,0.838495,1.823994,4.077145,3.920353
6858,71.201188,0.309349,14.953838,0.610951,2.219250,0.043731,0.839095,1.822565,4.078344,3.921690
...,...,...,...,...,...,...,...,...,...,...
201964,72.008082,0.281148,14.718623,0.763074,1.809972,0.043811,0.661319,1.395502,4.114014,4.204455
201965,72.020749,0.280544,14.714449,0.762675,1.806288,0.043782,0.658888,1.390632,4.113993,4.207999
201966,72.032166,0.279999,14.710676,0.762305,1.802976,0.043756,0.656701,1.386253,4.113971,4.211196
201967,72.042219,0.279518,14.707345,0.761973,1.800063,0.043734,0.654779,1.382404,4.113950,4.214015


In [83]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45001:90000]

In [84]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA1_Tr/area1_Tr_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA1_Tr/area1_Tr_2.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [85]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1_Tr/area1_Tr_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1_Tr/area1_Tr_2_results_excel.txt")

In [86]:
df_final = pd.concat([df_final1, df_final2])

In [87]:
df_final

,Q,P,K
SAMPLE,,,
6854,27.181803,38.352673,27.802908
6855,25.895496,40.107260,26.584814
6856,25.893578,40.099378,26.602108
6857,25.888922,40.095834,26.615573
6858,25.880690,40.097879,26.624507
...,...,...,...
201964,26.956204,38.250960,28.325613
201965,26.971991,38.226650,28.346515
201966,26.986174,38.204767,28.365407


## QAPF classification

In [88]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [89]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite',
       'quartz monzodiorite\nquartz monzogabbro'], dtype=object)

In [90]:
df_final.to_excel("../_CIPW/CIPW/AREA1_Tr/qapf.xlsx", index=True)

-------

In [91]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA1_Tr/QAPF_counts.xlsx")

-----

In [92]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area1_Tr.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [93]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control
QAPF_control.to_excel("../_CIPW/CIPW/AREA1_Tr/QAPF_control_points.xlsx", index=True)